# Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 16})

In [2]:
from tensorflow.keras.layers import Dense,Flatten, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

In [3]:
import tensorflow.keras.backend as K
K.clear_session()

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12509140601063910293
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3663069184
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17493795799658096013
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


# Image Preprocssing

In [5]:
image_height, image_width = (412, 412)
batch_size = 32

train_data_dir = 'train/'
test_data_dir = 'test/'

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  horizontal_flip=0.5,
                                  shear_range=0.5)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale')

Found 4035 images belonging to 2 classes.


In [7]:
test_generator = train_datagen.flow_from_directory(
                    test_data_dir,
                    target_size=(image_height, image_width),
                    batch_size=batch_size,
                    class_mode='categorical',
                    color_mode='grayscale')

Found 996 images belonging to 2 classes.


# Creating Model for choosing best model

In [8]:
x, y = train_generator.next()

In [9]:
model = Sequential()

model.add(Conv2D(32, (3, 3),
                 activation='relu',
                 input_shape=x.shape[1:]))

model.add(MaxPool2D(pool_size=(2, 2)))

model.add(BatchNormalization())

model.add(Conv2D(32, (3, 3), activation='relu'))

model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3),
                 activation='relu',
                 input_shape=x.shape[1:]))

model.add(MaxPool2D(pool_size=(2, 2)))

model.add(BatchNormalization())

model.add(Conv2D(32, (3, 3),
                 activation='relu',
                 input_shape=x.shape[1:]))

model.add(MaxPool2D(pool_size=(2, 2)))

model.add(BatchNormalization())

model.add(Flatten())

model.add(BatchNormalization())

model.add(Dropout(0.5))

model.add(Dense(512, activation = 'relu'))

model.add(BatchNormalization())

model.add(Dense(2, activation='softmax'))

In [10]:
x.shape

(32, 412, 412, 1)

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
%load_ext tensorboard

In [13]:
import datetime
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf

# Set Folder for saving logs
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [14]:
checkpoint = ModelCheckpoint("best_model.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto', period=1)

In [15]:
steps_per_epoch =  train_generator.n // batch_size
validation_steps= test_generator.n //batch_size

In [16]:
model.fit(train_generator, epochs=10, callbacks=[tensorboard_callback, checkpoint], validation_data=test_generator, validation_steps=validation_steps)

Epoch 1/10
127/127 [==============================] - ETA: 0s - loss: 0.6222 - accuracy: 0.7586
Epoch 00001: val_accuracy improved from -inf to 0.39718, saving model to best_model.hdf5
127/127 [==============================] - 57s 389ms/step - loss: 0.6222 - accuracy: 0.7586 - val_loss: 1.3304 - val_accuracy: 0.3972
Epoch 2/10
127/127 [==============================] - ETA: 0s - loss: 0.3178 - accuracy: 0.8615
Epoch 00002: val_accuracy did not improve from 0.39718
127/127 [==============================] - 46s 363ms/step - loss: 0.3178 - accuracy: 0.8615 - val_loss: 3.6706 - val_accuracy: 0.3952
Epoch 3/10
127/127 [==============================] - ETA: 0s - loss: 0.2365 - accuracy: 0.9046
Epoch 00003: val_accuracy did not improve from 0.39718
127/127 [==============================] - 46s 365ms/step - loss: 0.2365 - accuracy: 0.9046 - val_loss: 3.1956 - val_accuracy: 0.3972
Epoch 4/10
127/127 [==============================] - ETA: 0s - loss: 0.2416 - accuracy: 0.9024
Epoch 00004: va

In [17]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 1892), started 0:10:49 ago. (Use '!kill 1892' to kill it.)

In [18]:
model.evaluate(test_generator)

29/29 [==============================] - 9s 305ms/step - loss: 0.0475 - accuracy: 0.9868


[0.04749244451522827, 0.9868276715278625]

In [19]:
# import shutil

# shutil.rmtree('logs', ignore_errors=False)